In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns


# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

import env
import wrangle

In [2]:
df = wrangle.get_curriculum_logs()
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [3]:
df = wrangle.prep_curriculum_data()
df.head()

AttributeError: 'DataFrame' object has no attribute 'date'

# Explore

### Question 1: Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [ ]:
# curriculum access value counts per program

df.program.value_counts()


In [ ]:
# top 5 overall pages
print('10 Most Popular Pages in Web Dev Program & DS')

df.groupby(['program'])['endpoint'].value_counts().head(10)

In [ ]:
# remove pages that are not part of curriculum pages

df = df[df.endpoint.str.contains('jpeg') != True]

df = df[df.endpoint.str.contains('json') != True]

df = df[df.endpoint.str.contains('jpg') != True]

df = df[df.endpoint.str.contains('appendix') != True]

df = df[df.endpoint.str.contains('Appendix') != True]

df = df[df.endpoint != '/']

df = df[df.endpoint != 'toc']

In [ ]:
# page views per cohort with count
page_views = df.groupby(['cohort_name'])['endpoint'].agg(['count', 'unique'])
page_views

In [ ]:
# create ds dataframe
DS_df = df[(df.program == 'Data Science')]
DS_df.head(5)

In [ ]:
# top 5 overall pages
print('5 Most Popular Pages in Data Science Program')

DS_df.endpoint.value_counts().head(5)

In [ ]:
# create Web dev df
Wb_df = df[(df.program == 'Web Development')]
Wb_df.head(5)

In [ ]:
# plot data for top visited page 
plt.figure(figsize=(10,7))
Wb_df[Wb_df.endpoint == 'classification/overview'].endpoint.resample('Q').count().plot()
plt.title('Page visits of classification/overview')
plt.xlabel('Quarter')
plt.ylabel('Page Visits')

In [ ]:
# plot data for lowest of top 5 visited page
plt.figure(figsize=(10,7))
Wb_df[Wb_df.endpoint == 'fundamentals/intro-to-data-science'].endpoint.resample('Q').count().plot()
plt.title('Page visits of fundamentals/intro-to-data-science')
plt.xlabel('Quarter')
plt.ylabel('Page Visits')

In [ ]:
# top 5 overall pages
print('5 Most Popular Pages in Web Development Program')

Wb_df.endpoint.value_counts().head(5)

In [ ]:
# plot data for top visited page 

plt.figure(figsize=(10,7))
Wb_df[Wb_df.endpoint == 'javascript-i'].endpoint.resample('Q').count().plot()
plt.title('Page visits of javascript-i')
plt.xlabel('Quarter')
plt.ylabel('Page Visits')

In [ ]:
# plot data for lowest of top 5 visited page

plt.figure(figsize=(10,7))
Wb_df[Wb_df.endpoint == 'spring'].endpoint.resample('Q').count().plot()
plt.title('Page visits of Spring')
plt.xlabel('Quarter')
plt.ylabel('Page Visits')

### Takeaways
- For Data Science Program these are the top 5 pages with the most traffic:
    - classification/overview                    | count: 1785
    - 1-fundamentals/1.1-intro-to-data-science   | count: 1633
    - classification/scale_features_or_not.svg   | count: 1590
    - sql/mysql-overview                         | count: 1424
    - fundamentals/intro-to-data-science         | count: 1413
    
- For Web Development these are the top 5 pages with the most traffic:
    - javascript-i    | count: 18193
    - java-iii        | count: 13162
    - html-css        | count: 13111
    - java-ii         | count: 12173
    - spring          | count: 11877

In [ ]:
subset = df[(df.endpoint != '/') & (df.cohort_name != 'Staff') & (df.program != 'none')]


In [ ]:
subset[['base', 'endpoint']] = subset.endpoint.str.split(pat='/', n=1, expand=True)


In [ ]:
subset.base = subset.base.str.lstrip('0123456789.- ')
subset.head()

In [ ]:
ds_curr = subset[subset.program =='Web Development'].base.value_counts().sort_values(ascending=False).to_frame()
ds_curr.head(10)

In [ ]:
ds_curr = subset[subset.program =='Data Science'].base.value_counts().sort_values(ascending=False).to_frame()
ds_curr.head(10)

### 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
pd.DataFrame(DS_df.endpoint.value_counts().head(5)).index


In [ ]:
DS_views = DS_df[(DS_df.endpoint == 'classification/overview') | (DS_df.endpoint == '1-fundamentals/1.1-intro-to-data-science') | (DS_df.endpoint == 'classification/scale_features_or_not.svg') | (DS_df.endpoint == 'sql/mysql-overview') | (DS_df.endpoint == 'fundamentals/intro-to-data-science')]


In [ ]:
DS_views = pd.crosstab(DS_views.cohort_id, DS_views.endpoint)


In [ ]:
DS_views.columns = ['Classification/Overview','Fundamentals','Classification/Scaling','SQL Overview','Fundamentals/Intro to DS']


In [ ]:
x = 1
plt.figure(figsize=(13,8))
for col in DS_views.columns:
    plt.subplot(2,3,x)
    x = x + 1
    sns.barplot(x=DS_views.index, y=DS_views[col])
    plt.title(col, fontsize=18)
    plt.xlabel("Cohort ID")
    plt.ylabel("Page Views")
    
plt.tight_layout()


In [ ]:
# least referred to lessons across all cohorts for Web Development program

ds_curr = subset[subset.program =='Web Development'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

In [ ]:
# least referred to lessons across all cohorts for Data Science program

ds_curr = subset[subset.program =='Data Science'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

In [ ]:
# cohort Andromeda
ds_curr = subset[subset.cohort_name =='Andromeda'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

In [ ]:
# Cohort Apex
ds_curr = subset[subset.cohort_name =='Apex'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [ ]:
df.user_id.nunique()


In [ ]:
df.user_id.value_counts()

In [ ]:
page_views = df.groupby(['user_id'])['endpoint'].agg(['count', 'unique'])
page_views